In [ ]:
import pandas as pd
import numpy as np
import json
from datetime import datetime

# File paths
movies_df_path = r"E:\1_Project\recommender\data\content_based\v1\tmdb\tmdb_5000_movies.csv"
credits_df_path = r"E:\1_Project\recommender\data\content_based\v1\tmdb\tmdb_5000_credits.csv"

# Load data
movies_df = pd.read_csv(movies_df_path)
credits_df = pd.read_csv(credits_df_path)

# Step 1: Create 'credits' column by combining 'cast' and 'crew' from credits_df
new_credits_df = credits_df.copy()
new_credits_df['credits'] = new_credits_df.apply(lambda row: {'cast': row['cast'], 'crew': row['crew']}, axis=1)

# Keep only necessary columns
new_credits_df = new_credits_df[['movie_id', 'credits']]

# Step 2: Merge with movies_df
merged_df = pd.merge(movies_df, new_credits_df, left_on='id', right_on='movie_id', how='inner')
merged_df = merged_df.drop(columns=['movie_id'])

# Add media type and rename columns
merged_df['media_type'] = 'movie'
merged_df.rename(columns={'id': 'tmdb_id'}, inplace=True)

# Transformation function (same as yours)
def transform_movie_data(current_data):
    """Transform movie data from current structure to desired format."""
    def safe_parse(json_str, default=None):
        if json_str is None:
            return default or []
        if isinstance(json_str, list) or isinstance(json_str, dict):
            return json_str
        try:
            return json.loads(json_str.replace("'", '"'))
        except (json.JSONDecodeError, AttributeError):
            try:
                # Fix for broken JSON structure
                fixed_str = json_str.replace("}{", "},{").replace("][", "],[")
                if not fixed_str.startswith("["):
                    fixed_str = "[" + fixed_str
                if not fixed_str.endswith("]"):
                    fixed_str = fixed_str + "]"
                return json.loads(fixed_str)
            except:
                return default or []
    
    genres = safe_parse(current_data.get('genres'))
    keywords = safe_parse(current_data.get('keywords'))
    spoken_languages = safe_parse(current_data.get('spoken_languages'))
    production_companies = safe_parse(current_data.get('production_companies'))
    production_countries = safe_parse(current_data.get('production_countries'))
    
    # Handle credits properly - first get the credits dictionary
    credits = current_data.get('credits', {})
    
    # Parse cast and crew strings
    cast = safe_parse(credits.get('cast'))
    crew = safe_parse(credits.get('crew'))
    
    transformed_credits = []
    
    # Process directors from crew
    for person in crew:
        if person.get('job') == 'Director':
            transformed_credits.append({
                'type': 'director',
                'name': person.get('name'),
                'id': person.get('id'),
                'character': None,
                'image': f"https://image.tmdb.org/t/p/w500{person.get('profile_path', '')}" 
                         if person.get('profile_path') else None
            })
    
    # Process cast members
    for actor in cast:
        transformed_credits.append({
            'type': 'cast',
            'name': actor.get('name'),
            'id': actor.get('id'),
            'character': actor.get('character'),
            'image': f"https://image.tmdb.org/t/p/w500{actor.get('profile_path', '')}" 
                     if actor.get('profile_path') else None
        })
    
    release_date = current_data.get('release_date')
    if release_date:
        try:
            release_date = {'$date': datetime.strptime(release_date, '%Y-%m-%d').isoformat() + 'Z'}
        except (ValueError, TypeError):
            release_date = None
    
    transformed = {
        'data_status': 'Complete',
        'tmdb_id': current_data.get('tmdb_id'),
        'media_type': current_data.get('media_type', 'movie'),
        'title': current_data.get('title'),
        'original_title': current_data.get('original_title'),
        'overview': current_data.get('overview'),
        'genres': [{'id': g.get('id'), 'name': g.get('name')} for g in genres],
        'release_date': release_date,
        'runtime': current_data.get('runtime'),
        'vote_average': current_data.get('vote_average'),
        'vote_count': current_data.get('vote_count'),
        'poster_path': f"https://image.tmdb.org/t/p/w500{current_data.get('poster_path', '')}" 
                       if current_data.get('poster_path') else None,
        'backdrop_path': f"https://image.tmdb.org/t/p/original{current_data.get('backdrop_path', '')}" 
                         if current_data.get('backdrop_path') else None,
        'imdb_id': current_data.get('imdb_id'),
        'spoken_languages': [{
            'iso_639_1': lang.get('iso_639_1'),
            'name': lang.get('name')
        } for lang in spoken_languages],
        'release_status': current_data.get('status'),
        'tagline': current_data.get('tagline', ''),
        'homepage': current_data.get('homepage', ''),
        'revenue': current_data.get('revenue', 0),
        'budget': current_data.get('budget', 0),
        'adult': current_data.get('adult', False),
        'credits': transformed_credits,
        'trailer_id': None,
        'keywords': [{'id': kw.get('id'), 'name': kw.get('name')} for kw in keywords],
        'createdAt': {'$date': datetime.utcnow().isoformat() + 'Z'},
        'updatedAt': {'$date': datetime.utcnow().isoformat() + 'Z'},
        '__v': 0
    }
    
    return {k: v for k, v in transformed.items() if v is not None}

# Transform all movies
transformed_movies = [transform_movie_data(movie) for movie in merged_df.to_dict(orient='records')]

# Save to JSON file
with open(r"E:\1_Project\recommender\data\content_based\v1\coredb_media.json", 'w') as f:
    json.dump(transformed_movies, f, indent=2)

print(f"Successfully transformed and saved {len(transformed_movies)} movies")